# Sentiment Analysis with aspect and TEXT

a. Text [SEP] Aspect 

b. Text + Aspect

## importing libraries

In [ ]:
import json
from tqdm import tqdm
import pandas as pd
import numpy as np
import token

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import word_tokenize  

from collections import Counter
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import  confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


## Processing with training data 

In [ ]:
with open('/content/drive/My Drive/SentiHood/SentiHood Dataset/sentihood-train.json', 'r') as fp:
    training_set = json.load(fp)

In [ ]:
locations = ['LOCATION1']
aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety', 'shopping', 'touristy', 'transit-location']

for location in locations:

  df = pd.DataFrame({'id': [], 'text': [],'aspect':[], 'sentiment': []})

  ii = 0
  for each_example in training_set:
      id = str(int(each_example['id']))
      text = each_example['text'].strip()
      
      # If `location` is present in the text, only then iterate over the  
      # list of opinions to find suitable `location-aspect` datapoints.

      if location in text:
          aspect_found = False
          
          for opinion in each_example['opinions']:
              # Checking if the current example contains a sentiment
              # related to `location-aspect`
              
              if opinion['target_entity'] == location:
                  df.loc[ii] = [id, text,opinion['aspect'], opinion['sentiment']]
                  aspect_found = True
                  ii += 1
                  break
          
          # If no sentiment is found for `location-asppect` in current 
          # example, then add a datapoint with None.
          
          if not aspect_found:
              df.loc[ii] = [id, text,opinion['aspect'], 'None']
              ii += 1
        

## Processing with validation data

In [ ]:
with open('/content/drive/My Drive/SentiHood/SentiHood Dataset/sentihood-dev.json', 'r') as fp:
    validation_set = json.load(fp)

In [ ]:
locations = ['LOCATION1']
aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety', 'shopping', 'touristy', 'transit-location']

for location in locations:

  df_valid = pd.DataFrame({'id': [], 'text': [],'aspect':[], 'sentiment': []})

  ii = 0
  for each_example in validation_set:
      id = str(int(each_example['id']))
      text = each_example['text'].strip()
      
      # If `location` is present in the text, only then iterate over the  
      # list of opinions to find suitable `location-aspect` datapoints.

      if location in text:
          aspect_found = False
          
          for opinion in each_example['opinions']:
              # Checking if the current example contains a sentiment
              # related to `location-aspect`
              
              if opinion['target_entity'] == location:
                  df_valid.loc[ii] = [id, text,opinion['aspect'], opinion['sentiment']]
                  aspect_found = True
                  ii += 1
                  break
          
          # If no sentiment is found for `location-asppect` in current 
          # example, then add a datapoint with None.
          
          if not aspect_found:
              df_valid.loc[ii] = [id, text,opinion['aspect'], 'None']
              ii += 1
        

        # df.to_csv('/content/drive/My Drive/SentiHood/Bert-single/TrainingData/' + str(location) + str(aspect) + '.csv', index=False)
        # print(f"{location}{aspect} DONE!\tLength = {ii}")

## Processing with Testing data

In [ ]:
with open('/content/drive/My Drive/SentiHood/SentiHood Dataset/sentihood-test.json', 'r') as fp:
    testing_set = json.load(fp)  

In [ ]:
locations = ['LOCATION1']
aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety', 'shopping', 'touristy', 'transit-location']

for location in locations:

  df_test = pd.DataFrame({'id': [], 'text': [],'aspect':[], 'sentiment': []})

  ii = 0
  for each_example in testing_set:
      id = str(int(each_example['id']))
      text = each_example['text'].strip()
      
      # If `location` is present in the text, only then iterate over the  
      # list of opinions to find suitable `location-aspect` datapoints.

      if location in text:
          aspect_found = False
          
          for opinion in each_example['opinions']:
              # Checking if the current example contains a sentiment
              # related to `location-aspect`
              
              if opinion['target_entity'] == location:
                  df_test.loc[ii] = [id, text,opinion['aspect'], opinion['sentiment']]
                  aspect_found = True
                  ii += 1
                  break
          
          # If no sentiment is found for `location-asppect` in current 
          # example, then add a datapoint with None.
          
          if not aspect_found:
              df_test.loc[ii] = [id, text,opinion['aspect'], 'None']
              ii += 1

## Mapping labels to apects and sentiments

In [ ]:
# Changing aspect to labels
label_encoder=LabelEncoder()
integer_category = label_encoder.fit_transform(df.aspect)
df['label_aspect']=pd.DataFrame(integer_category)

# Changing sentiment to numerical label
df['sentiment']=df['sentiment'].map({'None':0,'Negative':1,'Positive':2})

In [ ]:
# Changing aspect to labels
label_encoder=LabelEncoder()
integer_category = label_encoder.fit_transform(df_valid.aspect)
df_valid['label_aspect']=pd.DataFrame(integer_category)

# Changing sentiment to numerical label
df_valid['sentiment']=df_valid['sentiment'].map({'None':0,'Negative':1,'Positive':2})

In [ ]:
# Changing aspect to labels
label_encoder=LabelEncoder()
integer_category = label_encoder.fit_transform(df_test.aspect)
df_test['label_aspect']=pd.DataFrame(integer_category)

# Changing sentiment to numerical label
df_test['sentiment']=df_test['sentiment'].map({'None':0,'Negative':1,'Positive':2})

In [ ]:
df_test.head()

,id,text,aspect,sentiment,label_aspect
0,153,LOCATION1 is in Greater London and is a very ...,safety,2,8
1,1130,All the neighborhoods around LOCATION1 are ver...,general,2,1
2,1271,"Cheap is LOCATION2, LOCATION1, but not really ...",general,1,1
3,1089,Dont Try LOCATION1,general,1,1
4,731,Find the website for 'Museum in LOCATION1' (pa...,general,0,1


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [ ]:
df['combined']=df['text']+" [SEP] "+df['aspect']

In [ ]:
df_valid['combined']=df_valid['text']+" [SEP] "+df_valid['aspect']

In [ ]:
df_test['combined']=df_test['text']+" [SEP] "+df_test['aspect']

In [ ]:
print(df['combined'].head(5))

0    LOCATION1 is transforming and the prices will go up and up [SEP] price                                                            
1    Along LOCATION1 there are lots of Electronics shops (independent ones) [SEP] shopping                                             
2    And LOCATION1 is ten mins direct on the tube to LOCATION2: [SEP] transit-location                                                 
3    Another option is LOCATION1 which is very central and has tons of clubs/bars within walking distance of each other [SEP] nightlife
4    Best bet is around LOCATION2 and LOCATION1 area in the northwest corner really nice [SEP] general                                 
Name: combined, dtype: object


## Model Buidling

### i. Aspect [SEP] Text

In [ ]:
 # importing transformers
 !pip install transformers

     |████████████████████████████████| 2.5MB 27.7MB/s 
     |████████████████████████████████| 3.3MB 29.1MB/s 
     |████████████████████████████████| 901kB 45.1MB/s 


In [ ]:
## pretrained model 
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
# Tokenize train, validation and test data
train_encoding= tokenizer(df['combined'].tolist(), padding="max_length",max_length=128, truncation=True)
valid_encoding= tokenizer(df_valid['combined'].tolist(), padding="max_length",max_length=128, truncation=True)
test_encoding= tokenizer(df_test['combined'].tolist(), padding="max_length",max_length=128, truncation=True)

In [ ]:
df['combined'][0]

'LOCATION1 is transforming and the prices will go up and up [SEP] price'

In [ ]:
## TensorFlow
import tensorflow as tf

# Tokens to tensor slices
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encoding),
    df['sentiment']
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(valid_encoding),
    df_valid['sentiment']
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encoding),
    df_test['sentiment']
))

In [ ]:
# Pre-training the Model
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=3)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_19', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
# Fine-tuning the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=3,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

In [ ]:
#Fine-tuning Model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(32),
          epochs=3,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/3
94/94 [==============================] - 50s 452ms/step - loss: 0.1721 - accuracy: 0.9476 - val_loss: 0.4781 - val_accuracy: 0.8461
Epoch 2/3
94/94 [==============================] - 41s 433ms/step - loss: 0.1092 - accuracy: 0.9694 - val_loss: 0.5102 - val_accuracy: 0.8501
Epoch 3/3
94/94 [==============================] - 41s 432ms/step - loss: 0.0726 - accuracy: 0.9829 - val_loss: 0.5220 - val_accuracy: 0.8554


In [ ]:
# final accuracy of the model 
accuracy_score(label , df_test['sentiment'])

0.8262910798122066

### ii. Aspect + Text

In [ ]:
# Combining Aspect + Text
df['combined_text']=df['text']+df['aspect']
df_valid['combined_text']=df_valid['text']+df_valid['aspect']
df_test['combined_text']=df_test['text']+df_test['aspect']

In [ ]:
# Tokenizing train, validation and test data 
train_encoding= tokenizer(df['combined_text'].tolist(), padding="max_length",max_length=128, truncation=True)
valid_encoding= tokenizer(df_valid['combined_text'].tolist(), padding="max_length",max_length=128, truncation=True)
test_encoding= tokenizer(df_test['combined_text'].tolist(), padding="max_length",max_length=128, truncation=True)

In [ ]:
## TensorFlow
import tensorflow as tf

# tokens to tensor slices
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encoding),
    df['sentiment']
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(valid_encoding),
    df_valid['sentiment']
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encoding),
    df_test['sentiment']
))

In [ ]:
# fine-tuning the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=3,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/3
187/187 [==============================] - 52s 236ms/step - loss: 0.1806 - accuracy: 0.9416 - val_loss: 0.6532 - val_accuracy: 0.8059
Epoch 2/3
187/187 [==============================] - 43s 230ms/step - loss: 0.0946 - accuracy: 0.9725 - val_loss: 0.6820 - val_accuracy: 0.8032
Epoch 3/3
187/187 [==============================] - 43s 228ms/step - loss: 0.0702 - accuracy: 0.9809 - val_loss: 0.7294 - val_accuracy: 0.8153


In [ ]:
#predicting values
tf_batch = tokenizer(df_test['combined_text'].tolist(), max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()

In [ ]:
# final Accuracy of aspect + text
accuracy_score(label , df_test['sentiment'])

0.795439302481556